In [1]:
%%capture
!pip install PyPDF2
!pip install unidecode
!pip install langchain
!pip install langchain-community
!pip install chromadb
!pip install sentence-transformers

# Obtencion de Datos

In [2]:
import gdown
import os
import shutil

# Link con archivos sobre recetas
url = 'https://drive.google.com/drive/folders/1x_DXa3qQZtSaSGjXa3ntycesMABwn2eR?usp=sharing'
output_folder = 'Archivos_necesarios'

# Verifica si la carpeta ya existe
if os.path.exists(output_folder):
    print(f"La carpeta '{output_folder}' ya existe. No se descargará nuevamente.")
else:
    # Descarga la carpeta
    gdown.download_folder(url, quiet=True, output=output_folder)
    print(f"La carpeta '{output_folder}' fue descargada con éxito.")


La carpeta 'Archivos_necesarios' fue descargada con éxito.


# Carga de Datos

In [5]:
import pandas as pd
import pickle

# Carga de PDFs
unidad_1 = '/content/Archivos_necesarios/Unidad 1 - Extracción y Procesamiento de Texto.pdf'
unidad_2 = '/content/Archivos_necesarios/Unidad 2 - Representación Vectorial de Texto.pdf'
unidad_3 = '/content/Archivos_necesarios/Unidad 3 - Procesamiento del Lenguaje.pdf'
unidad_4 = '/content/Archivos_necesarios/Unidad 4 - Arquitecturas de Modelos de Lenguaje.pdf'
unidad_5 = '/content/Archivos_necesarios/Unidad 5 - Almacenamiento y Representación del Conocimiento.pdf'
unidad_6 = '/content/Archivos_necesarios/Unidad 6 - Chatbots y Sistemas de Diálogo.pdf'
unidad_7 = '/content/Archivos_necesarios/Unidad 7 - Agentes Autónomos y Sistemas Inteligentes - 2024.pdf'

pdfs = [unidad_1, unidad_2, unidad_3, unidad_4, unidad_5, unidad_6, unidad_7]

# Carga del archivo CSV
ejercicios = pd.read_csv('/content/Archivos_necesarios/ejercicios_nlp_con_embeddings.csv')

# Carga del grafo
with open("/content/Archivos_necesarios/grafo_nlp.pickle", "rb") as f:
    G = pickle.load(f)

# Preprocesamiento de texto, extracción de PDF, eliminación de stopwords y split de datos con LangChain

In [8]:
import re
import nltk
import PyPDF2
from unidecode import unidecode
from nltk.corpus import stopwords
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Descargar stopwords si no están ya descargadas
nltk.download('stopwords', quiet=True)

def eliminar_stopwords(texto):
    # Elimina las stopwords del texto
    stop_words = set(stopwords.words('spanish'))
    palabras = texto.split()
    return ' '.join([word for word in palabras if word.lower() not in stop_words])

def extract_text_from_pdf(pdf_path):
    # Extrae el texto de un archivo PDF
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

def process_pdf_text(pdf_path):
    # Procesa el texto extraído del PDF
    text = extract_text_from_pdf(pdf_path)

    # Convierte todo el texto a minúsculas para uniformidad
    text = text.lower()

    # Elimina acentos y caracteres diacríticos
    text = unidecode(text)

    # Elimina todos los caracteres que no sean letras, números o espacios
    # Esto ayuda a estandarizar el texto y eliminar puntuación y símbolos
    return re.sub(r'[^a-z0-9\s]', '', text)

def split_text_with_langchain(text, chunk_size=500, chunk_overlap=50):
    # Divide el texto en fragmentos más pequeños
    # chunk_size: define el tamaño máximo de cada fragmento en caracteres
    # chunk_overlap: determina cuántos caracteres se superponen entre fragmentos
    #                esto ayuda a mantener el contexto entre fragmentos
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    return text_splitter.split_text(text)

# Crear un diccionario global para almacenar los fragmentos sin stopwords
global fragments_sin_stopwords_dict
fragments_sin_stopwords_dict = {}

# Dividir todos los PDFs y procesa los fragmentos para eliminar stopwords
for i, pdf_path in enumerate(pdfs, 1):
    processed_text = process_pdf_text(pdf_path)
    fragments = split_text_with_langchain(processed_text)

    fragments_sin_stopwords = [eliminar_stopwords(fragment) for fragment in fragments]

    # Almacenar los fragmentos sin stopwords en el diccionario global
    fragments_sin_stopwords_dict[f'unidad_{i}'] = fragments_sin_stopwords

    print(f"\nFragmentos de la Unidad {i}:")
    for j, fragment in enumerate(fragments_sin_stopwords, 1):
        # Muestra una vista previa de cada fragmento (primeros 50 caracteres)
        print(f"Fragmento {j}: {fragment[:50]}...")

    # Comparación de longitudes
    print(f"\nComparación de longitudes para la Unidad {i}:")
    for j, (frag_con, frag_sin) in enumerate(zip(fragments, fragments_sin_stopwords), 1):
        print(f"Fragmento {j}: Con stopwords: {len(frag_con)} caracteres, Sin stopwords: {len(frag_sin)} caracteres")

# Un chunk_size mayor dará fragmentos más largos, lo que puede ser útil para mantener más contexto
# Un chunk_overlap mayor aumentará la superposición, ayudando a preservar la continuidad entre fragmentos


Fragmentos de la Unidad 1:
Fragmento 1: unidad 1 extraccion procesamiento texto 1 extracci...
Fragmento 2: puede extraer texto documentos texto incluye docum...
Fragmento 3: noticias foros discusion wikipedia mas redes socia...
Fragmento 4: electronico asi asuntos encabezadosbases datos bas...
Fragmento 5: analisis formatos pueden ser pdf epub mobi capitul...
Fragmento 6: ser abiertos facil manipulacion seccion veremos ma...
Fragmento 7: formatotxt formato archivo texto mas basico archiv...
Fragmento 8: codigo abrira archivo llamado archivotxt mismo dir...
Fragmento 9: vemos hacerlo codificacion diferente iso88591 lati...
Fragmento 10: codigo argumento encodingiso88591 dice python arch...
Fragmento 11: comprender mejor diferencia diferentes encodings v...
Fragmento 12: 9 t 116 74 10 195 161 c3 a1 11 s 115 73 12 63 3fen...
Fragmento 13: encoding formato html html significa lenguaje marc...
Fragmento 14: 4 m 109 6d 5 111 6f 6 espacio 32 20 7 101 65 8 s 1...
Fragmento 15: generalmente te

# Embeddings

In [9]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np

# Cargar el modelo BERT en español
modelo_es = BertModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
tokenizador_es = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# Función que obtiene embeddings para cada texto
def obtener_embeddings(fragmentos):
    embeddings = []
    for fragmento in fragmentos:
        tokens = tokenizador_es(fragmento, truncation=True, padding=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            outputs = modelo_es(**tokens)
        embedding_vector = outputs.last_hidden_state.mean(dim=1).squeeze()
        embeddings.append(embedding_vector.tolist())
    return embeddings

# Diccionario para almacenar los embeddings
embeddings_dict = {}

# Obtener embeddings para cada unidad en el diccionario fragments_sin_stopwords_dict
for unidad, fragmentos in fragments_sin_stopwords_dict.items():
    embeddings = obtener_embeddings(fragmentos)
    embeddings_dict[unidad] = embeddings

    print(f"\nEmbeddings de los fragmentos de {unidad}:")
    print(np.array(embeddings).shape)

# Ejemplo de cómo acceder a los embeddings de una unidad específica
if 'unidad_1' in embeddings_dict:
    print("\nEjemplo de embeddings para unidad_1:")
    print(np.array(embeddings_dict['unidad_1']))

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Embeddings de los fragmentos de unidad_1:
(183, 768)

Embeddings de los fragmentos de unidad_2:
(168, 768)

Embeddings de los fragmentos de unidad_3:
(184, 768)

Embeddings de los fragmentos de unidad_4:
(146, 768)

Embeddings de los fragmentos de unidad_5:
(188, 768)

Embeddings de los fragmentos de unidad_6:
(186, 768)

Embeddings de los fragmentos de unidad_7:
(66, 768)

Ejemplo de embeddings para unidad_1:
[[-0.06136499 -0.01831084 -0.28445303 ... -0.33445165  0.37146997
   0.27996817]
 [-0.17318347  0.03546011 -0.35713133 ...  0.00659316  0.10719322
   0.34588417]
 [-0.31970727  0.098729   -0.50264031 ...  0.19529559 -0.03763414
   0.30434793]
 ...
 [ 0.3184672  -0.02400614 -0.1042008  ... -0.24092136 -0.10237644
  -0.02467822]
 [-0.17425859 -0.06212563 -0.39995247 ... -0.17951293 -0.12630746
   0.19860025]
 [ 0.18321912  0.2390171  -0.24267104 ... -0.21993849  0.41398019
  -0.38186288]]


# Chromadb

In [10]:
import chromadb
import numpy as np

# Crear el cliente y la colección de Chroma
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="teoria")

# Unir todos los fragmentos en una sola lista
todos_los_fragmentos = []
todos_los_embeddings = []
ids = []

for unidad, fragmentos in fragments_sin_stopwords_dict.items():
    todos_los_fragmentos.extend(fragmentos)
    todos_los_embeddings.extend(embeddings_dict[unidad])
    ids.extend([f'{unidad}_id{i+1}' for i in range(len(fragmentos))])

# Comprobamos la cantidad total de fragmentos
print(f"Número total de fragmentos: {len(todos_los_fragmentos)}")

# Comprobamos que tenemos la misma cantidad de embeddings que de fragmentos
print(f"Número total de embeddings: {len(todos_los_embeddings)}")

# Comprobamos que tenemos la misma cantidad de IDs que de fragmentos
print(f"Número total de IDs: {len(ids)}")

# Convertimos los embeddings a una lista de listas (si no lo están ya)
embeddings_lista = [embedding.tolist() if isinstance(embedding, np.ndarray) else embedding for embedding in todos_los_embeddings]

# Agregamos los embeddings a chromaDB
collection.add(
    embeddings=embeddings_lista,
    documents=todos_los_fragmentos,
    ids=ids
)

print(f"Se han agregado {len(todos_los_fragmentos)} fragmentos a la colección de ChromaDB.")

Número total de fragmentos: 1121
Número total de embeddings: 1121
Número total de IDs: 1121
Se han agregado 1121 fragmentos a la colección de ChromaDB.


# Modelo de clasificación

In [24]:
# Preparar los datos de entrenamiento
prompts = [
    # Dataframe (ejercicios)
    "Dame un ejercicio de codificación de archivos",
    "Necesito una práctica sobre extracción de texto",
    "Muestra un ejercicio de OCR",
    "Busco un problema de dificultad media sobre procesamiento de lenguaje natural",
    "¿Hay algún ejercicio sobre tokenización?",
    "Quiero practicar con un ejercicio de análisis de sentimientos",
    "Necesito un ejercicio práctico sobre clasificación de textos",
    "Dame un problema para resolver sobre vectorización de palabras",
    "Busco un ejercicio difícil de reconocimiento de entidades nombradas",
    "¿Tienes algún ejercicio sobre stemming y lematización?",
    "Proporciona un ejercicio de preprocesamiento de texto",
    "Dame una tarea práctica de normalización de texto",
    "Necesito un ejercicio para practicar la eliminación de stopwords",
    "¿Hay algún problema sobre construcción de n-gramas?",
    "Quiero un ejercicio de análisis de frecuencia de palabras",
    "Busco una práctica de codificación one-hot para texto",
    "Dame un ejercicio para implementar TF-IDF",
    "Necesito una tarea sobre bag-of-words",
    "¿Tienes algún problema práctico de parsing?",
    "Proporciona un ejercicio de etiquetado POS",

    # Grafo (teoría y conceptos)
    "Explícame la relación entre tokenización y análisis sintáctico",
    "¿Cómo se conecta el procesamiento de lenguaje natural con el aprendizaje automático?",
    "Muestra el camino de aprendizaje para dominar NLP",
    "¿Qué conceptos están relacionados con el análisis de sentimientos?",
    "Describe la estructura de un pipeline de NLP",
    "¿Cuáles son los componentes principales del procesamiento de lenguaje natural?",
    "Explica la conexión entre word embeddings y redes neuronales",
    "¿Cómo se relacionan la lingüística computacional y el NLP?",
    "Muestra los prerequisitos para aprender procesamiento de lenguaje natural",
    "¿Qué técnicas de deep learning se aplican en NLP?",
    "Explica el concepto de parsing en NLP",
    "¿Cuál es la diferencia entre procesamiento de lenguaje natural y comprensión del lenguaje natural?",
    "Describe los diferentes niveles de análisis en NLP",
    "¿Qué papel juega la semántica en el procesamiento de lenguaje natural?",
    "Explica el concepto de desambiguación en NLP",
    "¿Cómo se relaciona la teoría de la información con el NLP?",
    "Describe los desafíos del procesamiento de lenguaje natural",
    "¿Qué es la generación de lenguaje natural y cómo se relaciona con NLP?",
    "Explica el concepto de modelo de lenguaje en NLP",
    "¿Cuál es la importancia de la pragmática en el procesamiento de lenguaje natural?",

    # ChromaDB (fragmentos de texto y teoría detallada)
    "Encuentra información detallada sobre el funcionamiento de BERT",
    "Busca fragmentos que expliquen la arquitectura Transformer",
    "Necesito teoría profunda sobre el modelo GPT",
    "Dame información detallada sobre técnicas avanzadas de análisis de sentimientos",
    "Busco explicaciones extensas sobre el funcionamiento de word2vec",
    "Encuentra fragmentos que describan en detalle el proceso de named entity recognition",
    "Necesito información completa sobre técnicas de resumen automático de texto",
    "Busca teoría detallada sobre modelos de lenguaje neuronales",
    "Dame explicaciones profundas sobre la traducción automática neuronal",
    "Encuentra fragmentos que describan técnicas avanzadas de question answering",
    "Busca información detallada sobre el funcionamiento de ELMo",
    "Necesito una explicación profunda del mecanismo de atención en NLP",
    "Encuentra fragmentos que describan en detalle las redes neuronales recurrentes en NLP",
    "Dame información completa sobre el proceso de fine-tuning en modelos de lenguaje",
    "Busco explicaciones detalladas sobre la arquitectura de las redes LSTM",
    "Encuentra teoría profunda sobre el uso de transformers en NLP",
    "Necesito información extensa sobre técnicas de transfer learning en procesamiento de lenguaje",
    "Busca fragmentos que expliquen en detalle el funcionamiento de los autoencoders en NLP",
    "Dame explicaciones detalladas sobre las técnicas de regularización en modelos de lenguaje",
    "Encuentra información completa sobre la evaluación de modelos de NLP",

    # Más ejemplos mixtos
    "¿Cómo puedo implementar un clasificador de texto básico?",
    "Explícame la diferencia entre supervisado y no supervisado en NLP",
    "Necesito un ejercicio para practicar web scraping de texto",
    "Busco información detallada sobre el funcionamiento de SpaCy",
    "Dame un ejemplo de cómo usar regex en procesamiento de texto",
    "¿Puedes explicarme qué es transfer learning en el contexto de NLP?",
    "Quiero un ejercicio para practicar la creación de un chatbot simple",
    "Encuentra información sobre las últimas tendencias en NLP",
    "Necesito un ejercicio para practicar la generación de texto con GPT",
    "Explícame en detalle cómo funciona la atención en los modelos Transformer",
    "¿Cómo se implementa un sistema de recomendación basado en texto?",
    "Dame ejemplos de aplicaciones de NLP en el mundo real",
    "Necesito información sobre cómo manejar textos multilingües en NLP",
    "Explícame las diferencias entre word2vec, GloVe y FastText",
    "¿Cómo se puede usar NLP para el análisis de redes sociales?",
    "Dame un ejercicio para practicar la detección de temas en un corpus de texto",
    "Busco información sobre el uso de grafos de conocimiento en NLP",
    "¿Cómo se puede implementar un corrector ortográfico usando NLP?",
    "Necesito un ejercicio para practicar la generación de resúmenes de texto",
    "Explícame cómo funciona la búsqueda semántica en NLP",
]

categorias = [
    "dataframe", "dataframe", "dataframe", "dataframe", "dataframe",
    "dataframe", "dataframe", "dataframe", "dataframe", "dataframe",
    "dataframe", "dataframe", "dataframe", "dataframe", "dataframe",
    "dataframe", "dataframe", "dataframe", "dataframe", "dataframe",
    "grafo", "grafo", "grafo", "grafo", "grafo",
    "grafo", "grafo", "grafo", "grafo", "grafo",
    "grafo", "grafo", "grafo", "grafo", "grafo",
    "grafo", "grafo", "grafo", "grafo", "grafo",
    "chromadb", "chromadb", "chromadb", "chromadb", "chromadb",
    "chromadb", "chromadb", "chromadb", "chromadb", "chromadb",
    "chromadb", "chromadb", "chromadb", "chromadb", "chromadb",
    "chromadb", "chromadb", "chromadb", "chromadb", "chromadb",
    "dataframe", "grafo", "dataframe", "chromadb", "dataframe",
    "chromadb", "dataframe", "chromadb", "dataframe", "chromadb",
    "dataframe", "grafo", "chromadb", "chromadb", "grafo",
    "dataframe", "grafo", "dataframe", "dataframe", "chromadb",
]

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.metrics import classification_report, make_scorer, accuracy_score

# Preparación de datos
X = prompts
y = categorias

# Crear un pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=20)),  # Usar menos características
    ('clf', MultinomialNB())  # Naive Bayes, un modelo simple
])

# Configurar la validación cruzada dejando uno fuera
cv = LeaveOneOut()

# Realizar validación cruzada
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

print("Puntuaciones de validación cruzada:", scores)
print("Precisión media: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# Función para obtener predicciones de validación cruzada
def get_cv_predictions(model, X, y, cv):
    y_pred = []
    for train_index, test_index in cv.split(X):
        X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
        y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
        model.fit(X_train, y_train)
        y_pred.extend(model.predict(X_test))
    return y_pred

# Obtener predicciones de validación cruzada
y_pred = get_cv_predictions(pipeline, X, y, cv)

# Imprimir informe de clasificación
print("\nInforme de clasificación:")
print(classification_report(y, y_pred))

# Entrenar el modelo final con todos los datos
pipeline.fit(X, y)

# Función para predecir nuevos prompts
def predecir_categoria(prompt):
    return pipeline.predict([prompt])[0]

# Ejemplo de uso
nuevo_prompt = "Dame un ejercicio de codificación de archivos"
categoria_predicha = predecir_categoria(nuevo_prompt)
print(f"\nPara el prompt: '{nuevo_prompt}'")
print(f"La categoría predicha es: {categoria_predicha}")

Puntuaciones de validación cruzada: [1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0.
 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0.
 1. 0. 1. 1. 0. 0. 1. 0.]
Precisión media: 0.78 (+/- 0.84)

Informe de clasificación:
              precision    recall  f1-score   support

    chromadb       0.70      0.78      0.74        27
   dataframe       0.89      0.86      0.88        29
       grafo       0.73      0.67      0.70        24

    accuracy                           0.78        80
   macro avg       0.77      0.77      0.77        80
weighted avg       0.78      0.78      0.78        80


Para el prompt: 'Dame un ejercicio de codificación de archivos'
La categoría predicha es: dataframe
